# Real-Time Inferencing Service в Azure ML

Цель лабораторной работы: 

- развертывание обученной ML модели, как __web-сервиса, работающего в режиме реального времени__ (Real-Time Inferencing Service)
- подключение и получение прогнозов от web-сервиса, работающего в режиме реального времени.

## Подготовка среды

Импорт необходимых модулей и проверка версии AzureML SDK:

In [1]:
import azureml.core
from azureml.core import Workspace, Model

# Check core SDK version number
print(f'SDK version: {azureml.core.VERSION}')

SDK version: 1.19.0


Получим конфигурацию эксперимента: 

In [5]:
%run core.py

config = get_experiment_config('lab_7A')
init_experiment(config)
experiment_dir = get_experiment_dir(config)

config

Experiment realtime-service-experiment was initialized successfully.


{'experiment_name': 'realtime-service-experiment',
 'working_subdir': 'realtime-service-experiment-lab',
 'service_name': 'diabetes-predict-service',
 'core': {'expriments_root_dir': 'experiments/',
  'datastore_name': 'winter_school_2020',
  'dataset_name': 'diabetes-data',
  'ml_cluster_name': 'aml-ws-cluster',
  'ml_model_name': 'diabetes-predict-model'}}

## Соединение со Azure ML Workspace

Устанавливаем соединение с Рабочей областью в Azure ML:

In [17]:
ws = Workspace.from_config()
print(f'Successfully connected to Workspace: {ws.name}.')

Successfully connected to Workspace: ai-in-cloud-workspace.


## Подготовка к развертыванию web-сервиса

Процесс развертывания включает в себя следующие шаги:

- Определите __inference-конфигурацию сервиса__, которая включает скрипт для прогноза и конфигурацию Среды и используемую ML моделm.
- Определите __конфигурацию развертывания__, которая определяет среду выполнения, в которой будет размещен web-сервис. В данном случае это служба [экземпляров контейнеров Azure](https://docs.microsoft.com/ru-ru/azure/container-instances/container-instances-overview).
- Разверните модель как веб-сервис.
- Проверьте состояние развернутого веб-сервиса.

### Получение ML модели

Получим список уже обученных и зарегистрированных в Azure ML моделей машинного обучения:

In [18]:
for model in Model.list(ws):
    print(f'{model.name} v{model.version}')
    
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print(f'\t {tag_name}: {tag}')
        
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print(f'\t {prop_name}: {prop}')
        
    print('\n')

diabetes-predict-model v2
	 Lab: 4B
	 AUC: 0.8576982541474856
	 Accuracy: 0.7876666666666666


diabetes-predict-model v1
	 Lab: 4B
	 AUC: 0.8576982541474856
	 Accuracy: 0.7876666666666666


amlstudio-covid19-service v1
	 CreatedByAMLStudio: true


amlstudio-covid19-service-pipe v1
	 CreatedByAMLStudio: true


amlstudio-covid19-spread-servi v1
	 CreatedByAMLStudio: true


amlstudio-pima-diabets-service v2
	 CreatedByAMLStudio: true


amlstudio-letter-recognition-s v1
	 CreatedByAMLStudio: true


amlstudio-pima-diabetes-model v1
	 CreatedByAMLStudio: true




Выберем ML модель, которую хотим развернуть как web-сервис (по умолчанию берется последняя версия модели):

In [19]:
model = ws.models[config['core']['ml_model_name']]
print(f'{model.name} v{model.version}')

diabetes-predict-model v2


## Скрипт для прогнозов

Создадим скрипт, который содержит методы необходимые для:

- `init()`: инициализации web-сервиса
- `run(raw_data)`: прогноза на новых данных.


In [20]:
%%writefile scripts/score_model.py


#%% Import libraries
import os
import json
import joblib
import numpy as np

from azureml.core.model import Model


def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment. Join this path with the filename of the model file.
    # It holds the path to the directory that contains the deployed model (./azureml-models/$MODEL_NAME/$VERSION).
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model.pkl')
    model = joblib.load(model_path)


def run(new_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(new_data)['data'])
    
    # Get a prediction from the model
    pred = model.predict(data)
    
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    
    predicted_classes = []
    for p in pred:
        predicted_classes.append(classnames[p])
        
    # Return the predictions as JSON
    return json.dumps(predicted_classes)


Overwriting scripts/score_model.py


Скопируем скрипт в директорию эксперимента:

In [22]:
!cp scripts/score_model.py $experiment_dir
!ls $experiment_dir

score_model.py	service_env.yml


### Среда для запуска web-сервиса

Выгрузим Среду, содержащую все необходимые для ML модели зависимости, в файл конфигурации `service_env.yml`, и выведем содержимое этого файла на экран:

In [23]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
service_env = CondaDependencies()
service_env.add_conda_package('scikit-learn')

# Save the environment config as a .yml file
service_env_config = f'{experiment_dir}/service_env.yml'
with open(service_env_config, 'w') as f:
    f.write(service_env.serialize_to_string())
    
print(f'Dependecies was saved to {service_env_config}')

# Print the .yml file
with open(service_env_config, 'r') as f:
    print(f.read())

Dependecies was saved to experiments/realtime-service-experiment-lab/service_env.yml
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- scikit-learn
channels:
- anaconda
- conda-forge



## Развертывание web-сервиса 

In [24]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig


inference_config = InferenceConfig(runtime='python',
                                   source_directory=experiment_dir,
                                   entry_script='score_model.py',
                                   conda_file='service_env.yml')

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)


service = Model.deploy(ws, config['service_name'], [model], inference_config, deployment_config)

service.wait_for_deployment(True)

print(f'Service state: {service.state}')

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Service state: Healthy


In [ ]:
print(service.get_logs())

Если Вам нужно внести изменения и выполнить повторное развертывание, то Вам может потребоваться удалить неработоспособную службу, используя следующий код: `service.delete()`

Найдите на портале Azure ML на вкладке Конечных точек (Endpoints) успешно развернутый web-сервис. Вы также можете получить имена веб-сервисов, выполнив следующий код:

In [26]:
for webservice_name in ws.webservices:
    print(webservice_name)

diabetes-predict-service
covid19-service


## Подключение к сервису

Подключимся к сервису и получим прогноз по новым данным:

In [27]:
import json

# ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
new_data = [
    [2,180,74,24,21,23.9091702,1.488172308,22]
    #[4,180,74,24,21,23.9091702,1.488172308,60]
]
print('Patient: {}'.format(new_data[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({'data': new_data})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data=input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes)

Patient: [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22]
['not-diabetic']


## Удаление сервиса

Удалите web-сервис в случае отсутствия необходмости в нем, чтобы не тратить лишние средства по подписке Azure, выполнив следующий код:

In [16]:
service.delete()

## Полезные ссылки

1. https://docs.microsoft.com/ru-ru/azure/machine-learning/how-to-deploy-and-where?tabs=azcli
2. https://docs.microsoft.com/ru-ru/azure/machine-learning/how-to-deploy-and-where?tabs=azcli#choose-a-compute-target 